In [1]:
#第十章 数据聚合与分组运算
#对数据集进行分组并对各组应用一个函数（无论是聚合还是转换），通常是数据分析工作中的重要环节。
#在将数据集加载、融合、准备好之后，通常就是计算分组统计或生成数据透视表。4

In [2]:
#关系型数据库和SQL能够如此流行的原因之一就是其能够方便地对数据进行连接、过滤、转换和聚合。
import pandas as pd
import numpy as np
df=pd.DataFrame({'key1':['a','a','b','b','a'],
                'key2':['one','two','one','two','one'],
                'data1':np.random.randn(5),
                'data2':np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,0.833165,-0.772842
1,a,two,-0.067824,-0.760871
2,b,one,0.676063,-0.274467
3,b,two,0.347115,-0.665884
4,a,one,0.042247,0.262572


In [4]:
#假设想要按key1进行分组，并计算data1列的平均值。
#访问data1，并根据key1调用groupby:
grouped=df['data1'].groupby(df['key1'])

In [5]:
grouped

In [6]:
#变量grouped是一个GroupBy对象。
#我们可以调用GroupBy的mean方法来计算分组平均值：
grouped.mean()

key1
a    0.269196
b    0.511589
Name: data1, dtype: float64

In [7]:
means=df['data1'].groupby([df['key1'],df['key2']]).mean()

In [8]:
means

key1  key2
a     one     0.437706
      two    -0.067824
b     one     0.676063
      two     0.347115
Name: data1, dtype: float64

In [9]:
#通过两个键对数据进行了分组，得到的Series具有一个层次化索引
means.unstack()

key2,one,two
key1,,
a,0.437706,-0.067824
b,0.676063,0.347115


In [10]:
#分组键均为Series。实际上，分组键可以是任何长度适当的数组
states=np.array(['Ohio','California','California','Ohio','Ohio'])

In [11]:
states

array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'], dtype='<U10')

In [12]:
years=np.array([2005,2005,2006,2005,2006])

In [13]:
df['data1'].groupby([states,years]).mean()

California  2005   -0.067824
            2006    0.676063
Ohio        2005    0.590140
            2006    0.042247
Name: data1, dtype: float64

In [14]:
#分组信息就位于相同的要处理DataFrame中。还可以将列名（可以是字符串、数字或其他Python对象）作为分组键：
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.269196,-0.423714
b,0.511589,-0.470175


In [15]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.437706 -0.255135
     two  -0.067824 -0.760871
b    one   0.676063 -0.274467
     two   0.347115 -0.665884

In [16]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [17]:
#注意，任何分组关键词中的缺失值，都会被从结果中除去。
#对分组进行迭代
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.833165 -0.772842
1    a  two -0.067824 -0.760871
4    a  one  0.042247  0.262572
b
  key1 key2     data1     data2
2    b  one  0.676063 -0.274467
3    b  two  0.347115 -0.665884


In [18]:
#对于多重键的情况，元组的第一个元素将会是由键值组成的元组：
for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.833165 -0.772842
4    a  one  0.042247  0.262572
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.067824 -0.760871
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.676063 -0.274467
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.347115 -0.665884


In [19]:
#可以对这些数据片段做任何操作。有一个你可能会觉得有用的运算：将这些数据片段做成一个字典：
pieces=dict(list(df.groupby('key1')))

In [20]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.676063,-0.274467
3,b,two,0.347115,-0.665884


In [21]:
#groupby默认是在axis=0上进行分组的，通过设置也可以在其他任何轴上进行分组。
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [22]:
grouped=df.groupby(df.dtypes,axis=1)

In [23]:
for dtype,group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.833165 -0.772842
1 -0.067824 -0.760871
2  0.676063 -0.274467
3  0.347115 -0.665884
4  0.042247  0.262572
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [24]:
#选取一列或列的子集
#对于由DataFrame产生的GroupBy对象，如果用一个（单个字符串）或一组（字符串）列名对其进行索引，就能实现
#选取部分列进行聚合的目的。
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

In [25]:
df['data1'].groupby(df['key1'])

In [26]:
df['data2'].groupby(df['key1'])

In [27]:
#对于大数据集，很可能只需对部分列进行聚合。
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.255135
     two  -0.760871
b    one  -0.274467
     two  -0.665884

In [28]:
df.groupby(['key1','key2'])['data2'].mean()

key1  key2
a     one    -0.255135
      two    -0.760871
b     one    -0.274467
      two    -0.665884
Name: data2, dtype: float64

In [29]:
s_grouped=df.groupby(['key1','key2'])['data2']

In [30]:
s_grouped

In [31]:
s_grouped.mean()

key1  key2
a     one    -0.255135
      two    -0.760871
b     one    -0.274467
      two    -0.665884
Name: data2, dtype: float64

In [32]:
#通过字典或Series进行分组除数组意外，分组信息还可以其他形式存在。
people=pd.DataFrame(np.random.randn(5,5),columns=['a','b','c','d','e'],
                   index=['Joe','Steve','Wes','Jim','Travis'])

In [33]:
people.iloc[2:3,[1,2]]=np.nan

In [34]:
people

,a,b,c,d,e
Joe,0.305896,-0.986178,0.716851,1.571838,1.191534
Steve,-0.491344,0.804947,-1.247431,1.424350,2.024040
Wes,-1.186050,NaN,NaN,0.124740,-0.376149
Jim,-0.830110,-0.957178,0.911216,-0.144715,0.623586
Travis,-0.988536,-0.282753,0.645375,0.378375,1.135525


In [35]:
#假设已知列的分组关系，并希望根据分组计算列的和：
mapping={'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}

In [36]:
#可以将这个字典传给groupby，来构造数组，但我们可以直接传递字典
by_column=people.groupby(mapping,axis=1)
by_column.sum()

,blue,red
Joe,2.288689,0.511252
Steve,0.176919,2.337642
Wes,0.124740,-1.562200
Jim,0.766501,-1.163702
Travis,1.023750,-0.135764


In [37]:
map_series=pd.Series(mapping)

In [38]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [39]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [40]:
#通过函数进行分组，比起使用字典或Series，使用Python函数是一种更原生的方法定义分组
#分组映射。任何被当做分组键的函数都会在各个索引值上被调用一次，其返回值就会被分组名称。
people.groupby(len).sum()

,a,b,c,d,e
3,-1.710265,-1.943356,1.628067,1.551863,1.438971
5,-0.491344,0.804947,-1.247431,1.424350,2.024040
6,-0.988536,-0.282753,0.645375,0.378375,1.135525


In [41]:
#将函数跟数组、列表、字典、Series混合使用也不是问题。
key_list=['one','one','one','two','two']
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -1.186050 -0.986178  0.716851  0.124740 -0.376149
  two -0.830110 -0.957178  0.911216 -0.144715  0.623586
5 one -0.491344  0.804947 -1.247431  1.424350  2.024040
6 two -0.988536 -0.282753  0.645375  0.378375  1.135525

In [42]:
#根据索引级别分组
#层次化索引数据集最方便的地方就在于它能够根据轴索引的一个级别进行聚合：
columns=pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],
                                  names=['cty','tenor'])

In [43]:
hier_df=pd.DataFrame(np.random.randn(4,5),columns=columns)

In [44]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.677289 -0.646942 -1.846198  0.459055  1.366828
1     -0.434151  2.476559 -0.901433 -0.961086  1.755991
2     -0.633536 -0.210387 -0.220739  1.048663 -0.357008
3     -1.392528 -0.222964  1.348829  0.160014 -1.067432

In [45]:
#要根据级别分组，使用level关键字传递级别序号或名字：
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [46]:
#10.2数据聚合
#可以使用自己发明的聚合运算，还可以调用分组对象上已经定义好的任何方法。
#例如：quantile可以计算Series或DataFrame列的样本分位数。
df

,key1,key2,data1,data2
0,a,one,0.833165,-0.772842
1,a,two,-0.067824,-0.760871
2,b,one,0.676063,-0.274467
3,b,two,0.347115,-0.665884
4,a,one,0.042247,0.262572


In [47]:
grouped=df.groupby('key1')

In [48]:
grouped

In [49]:
grouped['data1'].quantile(0.9)

key1
a    0.674981
b    0.643168
Name: data1, dtype: float64

In [50]:
grouped['data1'].sum()

key1
a    0.807588
b    1.023178
Name: data1, dtype: float64

In [51]:
#如果要使用你自己的聚合函数，只需将其传入aggregate或agg方法即可。
def peak_to_peak(arr):
    return arr.max()-arr.min()

In [52]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.900989,1.035414
b,0.328948,0.391417


In [53]:
#有些方法（如descirbe）也是可以用在这里的，即使严格来讲，他们并非聚合运算：
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.269196  0.491502 -0.067824 -0.012788  0.042247  0.437706   
b      2.0  0.511589  0.232601  0.347115  0.429352  0.511589  0.593826   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.833165   3.0 -0.423714  0.594371 -0.772842 -0.766857 -0.760871   
b     0.676063   2.0 -0.470175  0.276774 -0.665884 -0.568030 -0.470175   

                          
           75%       max  
key1                      
a    -0.249150  0.262572  
b    -0.372321 -0.274467

In [56]:
#面向列的多函数应用
tips=pd.read_csv(r'C:\Users\86139\Desktop\examples_data_for_data_analysis\tips.csv')

In [57]:
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
5,25.29,4.71,No,Sun,Dinner,4
6,8.77,2.00,No,Sun,Dinner,2
7,26.88,3.12,No,Sun,Dinner,4
8,15.04,1.96,No,Sun,Dinner,2
9,14.78,3.23,No,Sun,Dinner,2


In [58]:
tips['tip_pct']=tips['tip']/tips['total_bill']

In [59]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [60]:
#对Series或DataFrame列的聚合运算其实就是使用aggregate（使用自定义函数）
#或调用诸如mean、std之类的方法。
grouped=tips.groupby(['day','smoker'])

In [61]:
grouped_pct=grouped['tip_pct']

In [62]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [63]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [64]:
grouped_pct.agg([('foo','mean'),('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [65]:
functions=['count','mean','max']

In [66]:
result=grouped['tip_pct','total_bill'].agg(functions)

In [67]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [68]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [69]:
ftuples=[('Durchnitt','mean'),['Abweichung',np.var]]

In [70]:
grouped['tip_pct','total_bill'].agg(ftuples)

tip_pct            total_bill            
            Durchnitt Abweichung  Durchnitt  Abweichung
day  smoker                                            
Fri  No      0.151650   0.000791  18.420000   25.596333
     Yes     0.174783   0.002631  16.813333   82.562438
Sat  No      0.158048   0.001581  19.661778   79.908965
     Yes     0.147906   0.003767  21.276667  101.387535
Sun  No      0.160113   0.001793  20.506667   66.099980
     Yes     0.187250   0.023757  24.120000  109.046044
Thur No      0.160298   0.001503  17.113111   59.625081
     Yes     0.163863   0.001551  19.190588   69.808518

In [71]:
grouped.agg({'tip':np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [72]:
grouped.agg({'tip_pct':['min','max','mean','std'],'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [73]:
#以‘没有行索引’的形式返回聚合数据
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [74]:
#10.3apply:一般性的‘拆分-应用-合并’
#最通用的GroupBy方法是apply，本节剩余部分重点讲解它。
def top(df,n=5,column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [75]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [76]:
#top函数在DataFrame的各个片段上调用
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [77]:
result=tips.groupby('smoker')['tip_pct'].describe()

In [78]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [79]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [80]:
f=lambda x: x.describe()

In [81]:
grouped.apply(f)

total_bill        tip       size    tip_pct
day  smoker                                                   
Fri  No     count    4.000000   4.000000   4.000000   4.000000
            mean    18.420000   2.812500   2.250000   0.151650
            std      5.059282   0.898494   0.500000   0.028123
            min     12.460000   1.500000   2.000000   0.120385
            25%     15.100000   2.625000   2.000000   0.137239
            50%     19.235000   3.125000   2.000000   0.149241
            75%     22.555000   3.312500   2.250000   0.163652
            max     22.750000   3.500000   3.000000   0.187735
     Yes    count   15.000000  15.000000  15.000000  15.000000
            mean    16.813333   2.714000   2.066667   0.174783
            std      9.086388   1.077668   0.593617   0.051293
            min      5.750000   1.000000   1.000000   0.103555
            25%     11.690000   1.960000   2.000000   0.133739
            50%     13.420000   2.500000   2.000000   0.173913
            75%     18.665000   3.240000   2.000000   0.209240
            max     40.170000   4.730000   4.000000   0.263480
Sat  No     count   45.000000  45.000000  45.000000  45.000000
            mean    19.661778   3.102889   2.555556   0.158048
            std      8.939181   1.642088   0.784960   0.039767
            min      7.250000   1.000000   1.000000   0.056797
            25%     14.730000   2.010000   2.000000   0.136240
            50%     17.820000   2.750000   2.000000   0.150152
            75%     20.650000   3.390000   3.000000   0.183915
            max     48.330000   9.000000   4.000000   0.291990
     Yes    count   42.000000  42.000000  42.000000  42.000000
            mean    21.276667   2.875476   2.476190   0.147906
            std     10.069138   1.630580   0.862161   0.061375
            min      3.070000   1.000000   1.000000   0.035638
            25%     13.405000   2.000000   2.000000   0.091797
            50%     20.390000   2.690000   2.000000   0.153624
...                       ...        ...        ...        ...
Sun  No     std      8.130189   1.224785   1.032674   0.042347
            min      8.770000   1.010000   2.000000   0.059447
            25%     14.780000   2.000000   2.000000   0.139780
            50%     18.430000   3.020000   3.000000   0.161665
            75%     25.000000   3.920000   4.000000   0.185185
            max     48.170000   6.000000   6.000000   0.252672
     Yes    count   19.000000  19.000000  19.000000  19.000000
            mean    24.120000   3.516842   2.578947   0.187250
            std     10.442511   1.261151   0.901591   0.154134
            min      7.250000   1.500000   2.000000   0.065660
            25%     17.165000   3.000000   2.000000   0.097723
            50%     23.100000   3.500000   2.000000   0.138122
            75%     32.375000   4.000000   3.000000   0.215325
            max     45.350000   6.500000   5.000000   0.710345
Thur No     count   45.000000  45.000000  45.000000  45.000000
            mean    17.113111   2.673778   2.488889   0.160298
            std      7.721728   1.282964   1.179796   0.038774
            min      7.510000   1.250000   1.000000   0.072961
            25%     11.690000   1.800000   2.000000   0.137741
            50%     15.950000   2.180000   2.000000   0.153492
            75%     20.270000   3.000000   2.000000   0.184843
            max     41.190000   6.700000   6.000000   0.266312
     Yes    count   17.000000  17.000000  17.000000  17.000000
            mean    19.190588   3.030000   2.352941   0.163863
            std      8.355149   1.113491   0.701888   0.039389
            min     10.340000   2.000000   2.000000   0.090014
            25%     13.510000   2.000000   2.000000   0.148038
            50%     16.470000   2.560000   2.000000   0.153846
            75%     19.810000   4.000000   2.000000   0.194837
            max     43.110000   5.000000   4.000000   0.241255

[64 rows x 4 columns]

In [82]:
#禁止分组键
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [88]:
#分位数和桶分析
#pandas有一些能根据指定面元或样本分位数将数据拆分成多块的工具（比如cut和qcut）。将这些函数跟groupby结合起来,就能轻松对数据集的桶（bucket）或分位数（quantile）分析了
frame=pd.DataFrame({'data1':np.random.randn(1000),'data2':np.random.randn(1000)})

In [89]:
quartiles=pd.cut(frame.data1,4)

In [90]:
quartiles[:10]

0    (0.0985, 1.577]
1    (-1.38, 0.0985]
2    (-1.38, 0.0985]
3    (-1.38, 0.0985]
4    (-1.38, 0.0985]
5    (0.0985, 1.577]
6    (-1.38, 0.0985]
7    (-1.38, 0.0985]
8    (0.0985, 1.577]
9    (0.0985, 1.577]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.864, -1.38] < (-1.38, 0.0985] < (0.0985, 1.577] < (1.577, 3.055]]

In [91]:
#由cut返回的categorical对象可直接传递到groupby
def get_stats(group):
    return {'min':group.min(),'max':group.max(),'count':group.count(),'mean':group.mean()}

In [92]:
grouped=frame.data2.groupby(quartiles)

In [93]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.864, -1.38]",73.0,2.667283,0.056094,-2.210919
"(-1.38, 0.0985]",441.0,2.470567,-0.054799,-2.806521
"(0.0985, 1.577]",427.0,2.691893,0.023984,-2.977513
"(1.577, 3.055]",59.0,2.098131,-0.027260,-2.729833


In [122]:
#示例：用特定于分组的值填充缺失值
#对于缺失数据的清洗工作，有时会用dropna将其换掉，而有时则可能会希望用一个固定值或数据集本身所衍生出来的值去填充Na值，这时就得使用fillna这个工具了
#用平均值去填充NA值
s=pd.Series(np.random.randn(6))
s

0   -0.291253
1   -0.948305
2   -0.653367
3    1.119554
4    0.672635
5   -1.317318
dtype: float64

In [124]:
s[::2]=np.nan

In [125]:
s

0         NaN
1   -0.948305
2         NaN
3    1.119554
4         NaN
5   -1.317318
dtype: float64

In [126]:
s.fillna(s.mean())

0   -0.382023
1   -0.948305
2   -0.382023
3    1.119554
4   -0.382023
5   -1.317318
dtype: float64

In [127]:
s

0         NaN
1   -0.948305
2         NaN
3    1.119554
4         NaN
5   -1.317318
dtype: float64

In [128]:
#假设需要对不同得分组填充不同的值。一种方法是将数据分组，并使用apply和能够对各数据快调用哦个fillna的函数即可。
states=['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']
group_key=['East']*4+['West']*4

In [129]:
data=pd.Series(np.random.randn(8),index=states)

In [130]:
data

Ohio          0.450451
New York      1.858667
Vermont      -1.060528
Florida      -0.105250
Oregon        0.337869
Nevada       -1.317942
California    0.128021
Idaho         0.297444
dtype: float64

In [131]:
data[['Vermont','Nevada','Idaho']]=np.nan

In [132]:
data

Ohio          0.450451
New York      1.858667
Vermont            NaN
Florida      -0.105250
Oregon        0.337869
Nevada             NaN
California    0.128021
Idaho              NaN
dtype: float64

In [133]:
data.groupby(group_key).mean()

East    0.734623
West    0.232945
dtype: float64

In [134]:
fill_mean=lambda g:g.fillna(g.mean())

In [135]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.450451
New York      1.858667
Vermont       0.734623
Florida      -0.105250
Oregon        0.337869
Nevada        0.232945
California    0.128021
Idaho         0.232945
dtype: float64

In [136]:
fill_values={'East':0.5,'West':-1}

In [137]:
fill_func=lambda g:g.fillna(fill_values[g.name])

In [138]:
data.groupby(group_key).apply(fill_func)

Ohio          0.450451
New York      1.858667
Vermont       0.500000
Florida      -0.105250
Oregon        0.337869
Nevada       -1.000000
California    0.128021
Idaho        -1.000000
dtype: float64

In [139]:
#示例：随机采样和排列
#假设想要从一个大数据集中随机抽取（进行替换或不替换）样本以进行蒙特卡罗模拟或其他分析工作。
#这里使用的方法是对Series使用sample方法
suits=['H','S','C','D']
card_val=(list(range(1,11))+[10]*3)*4

In [140]:
base_names=['A']+list(range(2,11))+['J','K','Q']
cards=[]

In [141]:
for suit in suits:
    cards.extend(str(num)+suit for num in base_names)

In [142]:
deck=pd.Series(card_val,index=cards)

In [145]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [146]:
def draw(deck,n=5):
    return deck.sample(n)

In [147]:
draw(deck)

JH     10
8H      8
6C      6
9D      9
10D    10
dtype: int64

In [148]:
#假设你想要从每种花色中随机抽取两张牌。由于花色是牌名的最后一块字符，所以我们可以据此进行分组，并使用apply：
get_suit=lambda card:card[-1]

In [149]:
deck.groupby(get_suit).apply(draw,n=2)

C  5C      5
   AC      1
D  2D      2
   3D      3
H  JH     10
   9H      9
S  10S    10
   4S      4
dtype: int64

In [150]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

7C      7
KC     10
10D    10
7D      7
10H    10
KH     10
9S      9
5S      5
dtype: int64

In [152]:
#示例：分组加权平均数和相关系数
#根据groupby的‘拆分-应用-合并‘范式，可以进行DataFrame的列与列之间或两个Series之间的运算（比如分组加权平均）
df=pd.DataFrame({'category':['a','a','a','a','b','b','b','b'],'data':np.random.randn(8),'weights':np.random.rand(8)})

In [153]:
df

,category,data,weights
0,a,0.681463,0.489129
1,a,-0.837271,0.385398
2,a,0.194806,0.766895
3,a,0.575159,0.143904
4,b,0.498706,0.857303
5,b,-0.277335,0.714795
6,b,0.055523,0.415662
7,b,1.353952,0.108886


In [154]:
grouped=df.groupby('category')

In [155]:
get_wavg=lambda g:np.average(g['data'],weights=g['weights'])

In [156]:
grouped.apply(get_wavg)

category
a    0.13600
b    0.19069
dtype: float64

In [159]:
#来自Yahoo！Finance的数据集，其中含有几只股票和标准普尔500指数（符号spx）的收盘价
close_px=pd.read_csv(r'C:\Users\86139\Desktop\examples_data_for_data_analysis\stock_px_2.csv',parse_dates=True,index_col=0)

In [160]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [162]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [170]:
#来做一个比较有趣的任务：计算一个由日收益率（通过百分数比变化计算）与spx之间的年度相关系数组成的DataFrame。下面是实现的一个方法，我们先创建一个函数，
#用它计算每列和Spx列的成对相关系数：
spx_corr=lambda x:x.corrwith(x['SPX'])

In [171]:
rets=close_px.pct_change().dropna()

In [172]:
get_year=lambda x:x.year

In [173]:
by_year=rets.groupby(get_year)

In [174]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [178]:
#还可以计算列与列之间的相关系数
by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [179]:
#示例:组级别的线性回归
import statsmodels.api as sm

In [182]:
def regress(data,yvar,xvars):
    Y=data[yvar]
    X=data[xvars]
    X['intercept']=1.
    result=sm.OLS(Y,X).fit()
    return result.params
    

In [183]:
#为了按年计算AAPL对APX收益率的线性回归，执行：
by_year.apply(regress,'AAPL',['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


In [184]:
#10.4透视表和交叉表
#dataFrame有一个pivot_table_table方法，还有一个顶级pandas.pivot_table函数。
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [186]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [187]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [188]:
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',aggfunc=len,margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [189]:
tips.pivot_table('tip_pct',index=['time','size','smoker'],columns='day',aggfunc='mean',fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

In [190]:
#交叉表：crosstab
#交叉表是一种用于计算分组频率的特殊透视表
data

Ohio          0.450451
New York      1.858667
Vermont            NaN
Florida      -0.105250
Oregon        0.337869
Nevada             NaN
California    0.128021
Idaho              NaN
dtype: float64

In [192]:
pd.crosstab([tips.time,tips.day],tips.smoker,margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [ ]:
#10.5下一章学时间序列